# Business Understanding

Linear Regression
* Fast
* Easily interpretable feature effect
* Easily evaluated
* Easily explained to non-technical
* Easily visualized

# Data Understanding

* Distribution
    * Range
    * Skew
* Correlation
* Visualization
* Data Types
    * Ints, Floats, Strings, Booleans
    * Numerical vs categorical?
* Missing data?

# Data Preparation

* Coerce any dtypes to proper format
* Normalize numerical
* Remove, or fill missing data
* One Hot Encode Categorical Data

# Modeling

# Evaluation

# Deployment

Putting it in a pipeline

Putting in a script.py and running it